## Exercise 7-2

## TransE

In this exercise we will study a latent distance model TransE. The TransE method models relationships by interpreting them as translations operating on the latent representation of entities.

The TransE model translates the latent representations via a relation-specific offset $r_k$:

$f_{ijk}^{TransE} = - d(a_i + r_k, a_j)$

where d(.) is a distance/dissimilarity function. If we consider d as a squared euclidean distance, we have:
$d(a_i + r_k, a_j) = ||a_i||_2^2 + ||r_k||_2^2 + ||a_j||_2^2 - 2(a_i^Ta_j + r_k^T(a_j-a_i))$

Considering unity norm constraints on $a_i, a_j$, ||a_i||_2^2 = ||r_k||_2^2 = 1, ||a_i||_2^2 + ||r_k||_2^2 = 2 is a constant so will not play role in optimization. THus final scoring function of TransE can be reduced to:

$f_{ijk}^{TransE} = - (||r_k||_2^2 - 2a_i^Ta_j + 2r_k^T(a_j-a_i))$

For evaluation of method we will use wordnet dataset. In wordnet entities correspond to word senses and relationships define lexical relations between them. 

In [55]:
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F
from torch.nn.init import xavier_normal,xavier_uniform
from torch.autograd import Variable
from utils import get_minibatches, sample_negatives, accuracy, auc
from time import time

In [72]:
class TransE(nn.Module):
    """
    TransE embedding model
    ----------------------
    Bordes, Antoine, et al.
    "Translating embeddings for modeling multi-relational data." NIPS. 2013.
    """

    def __init__(self, n_e, n_r, k, margin, distance='l2', gpu=False):
        """
        TransE embedding model
        ----------------------

        Params:
        -------
            n_e: int
                Number of entities in dataset.

            n_r: int
                Number of relationships in dataset.

            k: int
                Embedding size.

            margin: float
                Margin size for TransE's hinge loss.

            distance: {'l1', 'l2'}
                Distance measure to be used in the loss.

            gpu: bool, default: False
                Whether to use GPU or not.
        """
        super(TransE, self).__init__()
        # Parameters
        self.n_e = n_e
        self.n_r = n_r
        self.k = k
        self.gpu = gpu
        self.distance = distance
        self.gamma = margin
        # Embedding Layer
        self.emb_E = nn.Embedding(self.n_e, self.k)
        self.emb_R = nn.Embedding(self.n_r, self.k)
        # Initialization
        r = 6/np.sqrt(self.k)
        self.emb_E.weight.data.uniform_(-r, r)
        self.emb_R.weight.data.uniform_(-r, r)        
        # Copy all params to GPU if specified
        if self.gpu:
            self.cuda()

    def forward(self, X):
        X = Variable(torch.from_numpy(X)).long()
        X = X.cuda() if self.gpu else X
        # Decompose X into head, relationship, tail
        hs, ls, ts = X[:, 0], X[:, 1], X[:, 2]
        e_hs = self.emb_E(hs)
        e_ts = self.emb_E(ts)
        e_ls = self.emb_R(ls)
        f = self.energy(e_hs, e_ls, e_ts).view(-1, 1)
        return f

    def energy(self, h, l, t):
        if self.distance == 'l1':
            out = torch.sum(torch.abs(h + l - t), 1)
        else:
            out = torch.sqrt(torch.sum((h + l - t)**2, 1))
        return out
    
    def ranking_loss(self, y_pos, y_neg, C=1, average=True):
        """
        Compute loss max margin ranking loss.

        Params:
        -------
        y_pos: vector of size Mx1
            Contains scores for positive samples.

        y_neg: np.array of size Mx1 (binary)
            Contains the true labels.

        margin: float, default: 1
            Margin used for the loss.

        C: int, default: 1
            Number of negative samples per positive sample.

        average: bool, default: True
            Whether to average the loss or just summing it.

        Returns:
        --------
        loss: float
        """
        M = y_pos.size(0)

        y_pos = y_pos.view(-1).repeat(C) # repeat to match y_neg
        y_neg = y_neg.view(-1)
        target = Variable(torch.from_numpy(-np.ones(M*C, dtype=np.float32)))
        loss = nn.MarginRankingLoss(margin=self.gamma)
        loss = loss(y_pos, y_neg, target)
        return loss
    
    def normalize_embeddings(self):
        self.emb_E.weight.data.renorm_(p=2, dim=0, maxnorm=1)
    
    def predict(self, X, sigmoid=False):
        
        y_pred = self.forward(X).view(-1, 1)

        if sigmoid:
            y_pred = F.sigmoid(y_pred)

        if self.gpu:
            return y_pred.cpu().data.numpy()
        else:
            return y_pred.data.numpy()

In [73]:
# Set random seed
randseed = 9999
np.random.seed(randseed)
torch.manual_seed(randseed)

In [75]:
# Data Loading
# Load dictionary lookups
idx2ent = np.load('data/wordnet/bin/idx2ent.npy')
idx2rel = np.load('data/wordnet/bin/idx2rel.npy')

n_e = len(idx2ent)
n_r = len(idx2rel)

# Load dataset
X_train = np.load('data/wordnet/bin/train.npy')
X_val = np.load('data/wordnet/bin/val.npy')
y_val = np.load('data/wordnet/bin/y_val.npy')

X_val_pos = X_val[y_val.ravel() == 1, :]  # Take only positive samples

M_train = X_train.shape[0]
M_val = X_val.shape[0]

# Model Parameters
k = 50
distance = 'l2'
margin = 1.0
model = TransE(n_e=n_e, n_r=n_r, k=k, margin=margin, distance=distance, gpu= False)

In [77]:
normalize_embed = True
C = 5 # Negative Samples
n_epoch = 20
lr = 0.1
lr_decay_every = 20
#weight_decay = 1e-4
mb_size = 100  
print_every = 100
average = False
# Optimizer Initialization
#solver = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
solver = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# Begin training
for epoch in range(n_epoch):
    print('Epoch-{}'.format(epoch+1))
    print('----------------')
    it = 0
    # Shuffle and chunk data into minibatches
    mb_iter = get_minibatches(X_train, mb_size, shuffle=True)

    # Anneal learning rate
    lr = lr * (0.5 ** (epoch // lr_decay_every))
    for param_group in solver.param_groups:
        param_group['lr'] = lr

    for X_mb in mb_iter:
        start = time()

        # Build batch with negative sampling
        m = X_mb.shape[0]
        # C x M negative samples
        X_neg_mb = np.vstack([sample_negatives(X_mb, n_e) for _ in range(C)])
        X_train_mb = np.vstack([X_mb, X_neg_mb])

        y_true_mb = np.vstack([np.zeros([m, 1]), np.ones([C*m, 1])])

        # Training step
        y = model.forward(X_train_mb)
        y_pos, y_neg = y[:m], y[m:]
        loss = model.ranking_loss(y_pos, y_neg, C=C, average=average)        
        loss.backward()
        solver.step()
        solver.zero_grad()

        end = time()
        if normalize_embed:
            model.normalize_embeddings()

        end = time()
        # Training logs
        if it % print_every == 0:
            # Training auc
            pred = model.predict(X_train_mb, sigmoid=True)
            train_acc = auc(pred, y_true_mb)

            # Validation auc
            y_pred_val = model.forward(X_val)
            y_prob_val = F.sigmoid(y_pred_val)
            y_prob_val = 1 - y_prob_val
            val_acc = auc(y_prob_val.data.numpy(), y_val)

            print('Iter-{}; loss: {:.4f}; train_auc: {:.4f}; val_auc: {:.4f}; time per batch: {:.2f}s'
                    .format(it, loss.data[0], train_acc, val_acc, end-start))


        it += 1

Epoch-1
----------------
Iter-0; loss: 0.9417; train_auc: 0.5407; val_auc: 0.5543; time per batch: 0.04s
Iter-100; loss: 0.9033; train_auc: 0.5854; val_auc: 0.5569; time per batch: 0.03s
Iter-200; loss: 0.9010; train_auc: 0.5761; val_auc: 0.5597; time per batch: 0.03s
Iter-300; loss: 0.8848; train_auc: 0.6116; val_auc: 0.5627; time per batch: 0.04s
Iter-400; loss: 0.9339; train_auc: 0.5633; val_auc: 0.5663; time per batch: 0.03s
Iter-500; loss: 0.9163; train_auc: 0.5610; val_auc: 0.5691; time per batch: 0.03s
Iter-600; loss: 0.9208; train_auc: 0.5632; val_auc: 0.5717; time per batch: 0.04s
Iter-700; loss: 0.9123; train_auc: 0.5677; val_auc: 0.5740; time per batch: 0.03s
Iter-800; loss: 0.8922; train_auc: 0.5763; val_auc: 0.5768; time per batch: 0.04s
Iter-900; loss: 0.8581; train_auc: 0.5999; val_auc: 0.5792; time per batch: 0.04s
Iter-1000; loss: 0.8937; train_auc: 0.5782; val_auc: 0.5821; time per batch: 0.03s
Iter-1100; loss: 0.8893; train_auc: 0.5876; val_auc: 0.5841; time per batc

Iter-200; loss: 0.7269; train_auc: 0.6780; val_auc: 0.6215; time per batch: 0.04s
Iter-300; loss: 0.6577; train_auc: 0.7216; val_auc: 0.6207; time per batch: 0.04s
Iter-400; loss: 0.7000; train_auc: 0.6819; val_auc: 0.6201; time per batch: 0.04s
Iter-500; loss: 0.6926; train_auc: 0.6811; val_auc: 0.6188; time per batch: 0.03s
Iter-600; loss: 0.6795; train_auc: 0.6982; val_auc: 0.6191; time per batch: 0.03s
Iter-700; loss: 0.7166; train_auc: 0.6625; val_auc: 0.6189; time per batch: 0.03s
Iter-800; loss: 0.7275; train_auc: 0.6600; val_auc: 0.6185; time per batch: 0.04s
Iter-900; loss: 0.7627; train_auc: 0.6463; val_auc: 0.6181; time per batch: 0.03s
Iter-1000; loss: 0.7032; train_auc: 0.6801; val_auc: 0.6180; time per batch: 0.03s
Iter-1100; loss: 0.7196; train_auc: 0.6740; val_auc: 0.6174; time per batch: 0.04s
Epoch-10
----------------
Iter-0; loss: 0.7252; train_auc: 0.6498; val_auc: 0.6174; time per batch: 0.04s
Iter-100; loss: 0.7480; train_auc: 0.6506; val_auc: 0.6165; time per bat

Iter-400; loss: 0.6181; train_auc: 0.7179; val_auc: 0.6184; time per batch: 0.03s
Iter-500; loss: 0.5968; train_auc: 0.6902; val_auc: 0.6186; time per batch: 0.03s
Iter-600; loss: 0.6021; train_auc: 0.7294; val_auc: 0.6181; time per batch: 0.04s
Iter-700; loss: 0.6790; train_auc: 0.6874; val_auc: 0.6176; time per batch: 0.04s
Iter-800; loss: 0.6821; train_auc: 0.6712; val_auc: 0.6193; time per batch: 0.04s
Iter-900; loss: 0.5987; train_auc: 0.7077; val_auc: 0.6196; time per batch: 0.03s
Iter-1000; loss: 0.5910; train_auc: 0.7243; val_auc: 0.6188; time per batch: 0.04s
Iter-1100; loss: 0.6530; train_auc: 0.6818; val_auc: 0.6197; time per batch: 0.04s
Epoch-18
----------------
Iter-0; loss: 0.5960; train_auc: 0.7109; val_auc: 0.6197; time per batch: 0.03s
Iter-100; loss: 0.6049; train_auc: 0.7170; val_auc: 0.6192; time per batch: 0.04s
Iter-200; loss: 0.6292; train_auc: 0.7002; val_auc: 0.6194; time per batch: 0.03s
Iter-300; loss: 0.6033; train_auc: 0.7102; val_auc: 0.6194; time per bat

## Task: Load test data from wordnet folder and do the following analysis

(i) Use above trained model on test set and report auc.

(ii) Randomly select 10 entities from test set and display their 3-nearest neighbor.

(iii) For all triples $h_i,t_j,r_k$ define joint representation by computing $h_i+r_k-t_j$. Further do the t-sne plot for joint representation. 

In [ ]:
###########################
###### Your Code Here
###########################

## Task: Analysis on kinship dataset

Formulation of TransE can be seen as encoding a series of 2-way interactions. Thus for modeling data where 3-way dependencies are crucial, TransE might not work well. To understand this effect train and test TransE on the kinship dataset.


In [ ]:
###########################
###### Your Code Here
###########################